In [2]:
pip install tensorflow-datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [4]:
(train_ds, validation_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised= True,
    with_info= True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9


Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.G6M5X3_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [5]:
# PRE-PROCESSING THE IMAGE
def preprocess(image, label):
    image = tf.image.resize(image, [64, 64]) # resizing the images
    image = tf.cast(image, tf.float32)/255.0 # normalize pixel values, everything is between zero and one
    return image, label


In [6]:
def augument(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    return image, label

In [7]:
train_ds = train_ds.map(preprocess).map(augument).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE) #TRAIN DATASET
validation_ds = validation_ds.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE) # VALIDATION DATASET

In [8]:
# BUILDING CNN MODEL
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)), # 32 filters of size 3 X 3
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'), # 64 filters of size 3 X 3
    MaxPooling2D(2,2),
    Flatten(), # flatten everything
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid') # Binary classification, cat or dog
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# COMPILING THE MODEL
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# TRAINING THE MODEL
history = model.fit(train_ds, validation_data=validation_ds, epochs=10)
# EVALUATE THE MODEL
val_loss, val_accuracy = model.evaluate(validation_ds)
print(f"validation accuracy: {val_accuracy*100:.2f}")

Epoch 1/10


I0000 00:00:1727121967.724895     127 service.cc:145] XLA service 0x7fc100008dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727121967.724961     127 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727121967.724966     127 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 14/582 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.4639 - loss: 0.7879

I0000 00:00:1727121970.358862     127 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


582/582 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.5571 - loss: 0.6811 - val_accuracy: 0.6457 - val_loss: 0.6231
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.6713 - loss: 0.5990 - val_accuracy: 0.6948 - val_loss: 0.5742
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.7045 - loss: 0.5665 - val_accuracy: 0.7270 - val_loss: 0.5515
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.7285 - loss: 0.5379 - val_accuracy: 0.7360 - val_loss: 0.5355
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.7440 - loss: 0.5153 - val_accuracy: 0.7483 - val_loss: 0.5141
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.7586 - loss: 0.4927 - val_accuracy: 0.7537 - val_loss: 0.5044
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.7742 - loss: 0.4751 - val_accuracy: 0.7650 - val_loss: 0.4890
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.7855 - loss: 0.4576 - val_accurac